<a href="https://colab.research.google.com/github/kshitijahande/Hate-Detection/blob/main/2_classifier_tfidf_pos_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries
!pip install -qq ipynb
!pip install -qq plotly
!pip install -qq import-ipynb
!pip install -qq icecream
!pip install -qq transformers
!pip install -qq datasets


     |████████████████████████████████| 2.6 MB 23.5 MB/s 
     |████████████████████████████████| 3.3 MB 26.1 MB/s 
     |████████████████████████████████| 636 kB 35.0 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 264 kB 26.4 MB/s 
     |████████████████████████████████| 243 kB 43.0 MB/s 
     |████████████████████████████████| 118 kB 46.1 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 


In [2]:
#Import libraries
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random as rn
from icecream import ic 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import os


# Load Dataset from Hugging Face library

In [3]:

dataset = load_dataset('hate_speech_offensive', split='train')


Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset hate_speech_offensive downloaded and prepared to /root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5. Subsequent calls will reuse this data.


In [4]:
#Columns in dataset
ic(dataset[1]['tweet'])
ic(dataset[1])
ic(dataset.column_names)
ic(dataset.num_rows)

"""
Task 1 - multiclass classification
  "class":[
              0:"hate speech"
              1:"offensive language"
              2:"neither"
  ]
"""

ic| dataset[1]['tweet']: ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the '
                          '1st place!!')
ic| dataset[1]: {'class': 1,
                 'count': 3,
                 'hate_speech_count': 0,
                 'neither_count': 0,
                 'offensive_language_count': 3,
                 'tweet': '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe '
                          'in the 1st place!!'}
ic| dataset.column_names: ['count',
                           'hate_speech_count',
                           'offensive_language_count',
                           'neither_count',
                           'class',
                           'tweet']
ic| dataset.num_rows: 24783


24783

In [5]:
#Shuffle and split train and test dataset randomly
train_test_split = dataset.train_test_split(shuffle=True)

In [6]:
#Get train and set data from original dataset
test = train_test_split.get('test')
train = train_test_split.get('train')

#convert to data frame
test_df = test.to_pandas()
train_full_df = train.to_pandas()

# split train dev/validation set
train_df, val_df = train_test_split(train_full_df, test_size=0.2)



In [9]:
# Match total rows in each data frame
ic(train_df.shape[0])
ic(test_df.shape[0])
ic(val_df.shape[0])
total = train_df.shape[0] + test_df.shape[0] + val_df.shape[0]
ic(total)
ic(dataset.num_rows)

ic| train_df.shape[0]: 14869
ic| test_df.shape[0]: 6196
ic| val_df.shape[0]: 3718
ic| total: 24783
ic| dataset.num_rows: 24783


24783

In [10]:
# Hate vs non-hate distribution in data
train_df['class'].value_counts(ascending=True)


0      842
2     2513
1    11514
Name: class, dtype: int64

## FEATURE GENERATION

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
# Use tf-idf vectorizer

other_exclusions = ["#ff", "ff", "rt"]
stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

"""This function replace words to standardized counts 1) urls with URLHERE 2) lots of whitespace with one instance 3) mentions with MENTIONHERE"""

def preprocess(text_string):
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

"""This function removes punctuation & excess whitespace, sets to lowercase,and stems tweets. Returns a list of stemmed tokens."""

def tokenize(tweet):
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but no stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [14]:
# TFIDF matrix and scores
tfidf = vectorizer.fit_transform(train_df['tweet']).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys:indices; values:IDF scores

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [16]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in train_df['tweet']:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)


#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )


#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [17]:
#Now join all features
M = np.concatenate([tfidf,pos],axis=1)


In [19]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables

# TASK 1 - Train and evaluate classifier

In [20]:
X = pd.DataFrame(M)
y = train_df['class'].astype(int)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [22]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l2",C=0.01))
X_ = select.fit_transform(X,y)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_, y)

In [24]:
y_preds = model.predict(X_)


In [25]:
report = classification_report( y, y_preds )


In [26]:
print(report)


              precision    recall  f1-score   support

           0       0.60      0.79      0.68       842
           1       0.97      0.91      0.94     11514
           2       0.80      0.92      0.85      2513

    accuracy                           0.91     14869
   macro avg       0.79      0.87      0.82     14869
weighted avg       0.92      0.91      0.91     14869



In [27]:
from sklearn.metrics import f1_score
f1_report = f1_score(y, y_preds, average='weighted')
print (f1_report)

0.9108765714441177
